In [ ]:
from datetime import datetime
import os
transaction_file = open("transactions_SYB.txt", "a")
ingredient_file = open("sugar_ingred_level_SYB.txt", "a")
coin_file = open("coin_SYB.txt", "a")

##U-002

class Products:
    def __init__(self, name, price, time, count):
        self.name = name
        self.price = price
        self.time = time
        self.count = count

class VendingMachine:
    def __init__(self):
        self.items = [
            Products("espresso", 0.50, '20sec', 10),
            Products("long black", 1.00, '30sec', 10),
            Products("capuccino", 1.50, '50sec', 1),
            Products("latte", 1.00, '60sec', 0)
        ]
        self.coins = 0.00
        self.accepted_coins = [0.10, 0.20, 0.50, 1.00, 2.00]


    def display(self):
        for prod_id, item in enumerate(self.items, start=1):
##S-002
            if item.count > 0:
                print(f"{prod_id}. {item.name} --- ${item.price:.2f}")
        if item.count == 0:
            print(f"{item.name} --- sold out")

    def inserted_coin(self, money):
        if money <= 0.00:
            raise ValueError
        self.coins += money

#U-003, U-004
##S-001
class Status:
    def __init__(self, working_notworking):
        self.working_notworking = working_notworking
    def stat(self):
        print(f"The machine is {self.working_notworking}")

working = Status('working')
notworking = Status('not working')
working.stat()

##S-008
def admin_reset():
    os.remove("transactions_SYB.txt")
    main()

def restart():
        main()

##D-003, S-003
def sugar():
    while True:
        sugar_amount = 10
        add_sugar = input("Add sugar? (Y/N)")
        if add_sugar in 'Yy':
            sugar_amount -= 1
            print("Remaining sugar in machine: " + str(sugar_amount))
            ingredient_file.write("______________Current Sugar Level" + str(datetime.now()) + "\n")
            ingredient_file.write(f"The level of sugar: {sugar_amount}" + "\n")

            if sugar_amount < 1:
                print("sugar is unavailable")
                break
            print("Sugar will be added to your drink.")

            break
        elif add_sugar in 'Nn':
            print("No sugar will be added to your drink.")
            break
        else:
            print("Invalid input")

def main():
#S-005 1/2
    print("********************** Welcome to the vending machine *******************************")

    vm = VendingMachine()
    vm.display()

##S-009
    while True:
        try:
            choice = int(input("Please enter the product ID (digits only): "))
        except ValueError:
            continue
        if choice in range(1, len(vm.items)+1):
            break


#U-003
    item =vm.items[choice-1]
    print(f"You've selected \"{item.name}\" - the price is ${item.price:.2f}")
    confirm = input("Continue(Y / N)?")
    if confirm in 'Nn':
        print("No purchase made. Bye")
        restart()

##S-006
    elif confirm in 'Yy':
        print(f"{item.name} will take {item.time}")
##sugar was here
    else:
        print("Invalid input")

##S-007, ##S-011
    while vm.coins < item.price:
        print(f"Current balance: ${vm.coins:.2f}")
##U-004
        while True:
            try:
                inserted = float(input(
                    "How much would you like to insert? ($0.10, $0.20, $0.50, $1, $2 coins accepted)""\n"f"(The balance needs to be higher than the cost ${item.price:.2f} ):$"))
                coin_file.write("Coin received at date time: " + str(datetime.now()) + "\n")
                coin_file.write(f"Coin inserted: ${inserted:.2f}" + "\n")
                while inserted not in vm.accepted_coins:
                    print("Accepted coins only.")
                    inserted = float(input(
                        "How much would you like to insert? ($0.10, $0.20, $0.50, $1, $2 coins accepted)""\n"f"(The balance needs to be higher than the cost ${item.price:.2f} ):$"))
                while True:
                    more_coins = input("Insert more coin?(Y/N): ")
                    if more_coins in "Yy":
                        inserted = float(input(
                            "How much would you like to insert? ($0.10, $0.20, $0.50, $1, $2 coins accepted)""\n"f"(The balance needs to be higher than the cost ${item.price:.2f} ):$"))
                        vm.coins += inserted
                        coin_file.write("Coin received at date time: " + str(datetime.now()) + "\n")
                        coin_file.write(f"Coin inserted: ${inserted:.2f}" + "\n")

                        while inserted not in vm.accepted_coins:
                            print("Accepted coins only.")
                            inserted = float(input(
                                "How much would you like to insert? ($0.10, $0.20, $0.50, $1, $2 coins accepted)""\n"f"(The balance needs to be higher than the cost ${item.price:.2f} ):$"))
                            vm.coins -= inserted
                        print(f"Current balance: ${vm.coins + inserted:.2f}")
                    elif more_coins in "Nn":
                        if vm.coins + inserted < item.price:
                            reset = input("Not enough balance. Reset - R or Continue - C?")
                            if reset in "Rr":
                                print("restarting. please take your coins")
                                main()
                            if reset in "Cc":
                                break
                            else:
                                print("Invalid input")
                        break
                    else:
                        print("Invalid input")
                vm.inserted_coin(inserted)
            except ValueError:
                continue
            else:
                break

        ##U-001, #U-005, #U-007, #S-004, #S-007, #S-005 1/2
    while True:
        u_cancel = input("-Press C to cancel- or -# to change- or -R to restart- or -Y to dispense- : ")
        if u_cancel in 'Cc':
            print(f"please take your change ${vm.coins:.2f}. Goodbye.")
            main()
        if u_cancel == '#':
            print("Changing order: ")
            main()
        if u_cancel in 'Rr':
            print(f"Resetting and clearing transaction, your change ${vm.coins:.2f} is returned.")
            main()
        if u_cancel in 'Yy':
            sugar()
            print("Thank you.")
            print(f"Please take your \"{item.name}\".")
##D-006
            transaction_file.write("______________Transaction______________" + '\n')
            transaction_file.write(f"Date of sale:" + str(datetime.now()) + "\n")
            transaction_file.write(f"Product sold: {item.name}" + "\n")
            transaction_file.write(f"Sale amount: ${item.price:.2f}" + "\n")
            print(f"The remaining change in the machine is ${vm.coins - item.price:.2f}.")
            print("+++++++++++++++++++++++++++++\n" +
                    str(datetime.now()) + "\n" + f"Product sold: {item.name}\nSale amount: ${item.price:.2f}\n+++++++++++++++++++++++++++++")
            break
        else:
            print("Invalid input.")


    #U-006, #S-005 2/2
    while vm.coins > vm.coins - item.price:
        change_quit = input("Would you like to buy more items? (Y/N)")
        if change_quit in 'Nn':
            print(f"please take your change ${vm.coins - item.price:.2f}. Good Bye")

            system_reset = input("Admin only: To reset to factory setting - '*' or back to main menu - 'M': ")
            while True:
                if system_reset == '*':
                    with open("transactions_SYB.txt", 'w') as file:
                        pass
                    print("||||||||||||||||||||||||||||||| All Transactions Deleted From The System |||||||||||||||||||||||||||||||")
                    print("")
                    main()
                elif system_reset in 'Mn':
                    main()
                else:
                    break

        elif change_quit in 'Yy':
            vm = VendingMachine()
            transaction_file.write("______________Transaction______________" + '\n')
            transaction_file.write(f"Date of sale:" + str(datetime.now()) + "\n")
            transaction_file.write(f"Product sold: {item.name}" + "\n")
            transaction_file.write(f"Sale amount: ${item.price:.2f}" + "\n")
            vm.display()
            while True:
                try:
                    choice = int(input("Please enter the product ID (1 - 4 only): "))
                except ValueError:
                    continue
                if choice in range(1, len(vm.items) + 1):
                    break
            item = vm.items[choice - 1]
            print(f"You've selected \"{item.name}\" - the price is ${item.price:.2f}")
            sugar()
            vm.inserted_coin(inserted)


        else:
            print("Invalid input")

if __name__ == "__main__":
    import sys
    sys.exit(main())



The machine is working
********************** Welcome to the vending machine *******************************
1. espresso --- $0.50
2. long black --- $1.00
3. capuccino --- $1.50
latte --- sold out
Please enter the product ID (digits only): 3
You've selected "capuccino" - the price is $1.50
Continue(Y / N)?2
Invalid input
Current balance: $0.00
How much would you like to insert? ($0.10, $0.20, $0.50, $1, $2 coins accepted)
(The balance needs to be higher than the cost $1.50 ):$2
Insert more coin?(Y/N): Y
How much would you like to insert? ($0.10, $0.20, $0.50, $1, $2 coins accepted)
(The balance needs to be higher than the cost $1.50 ):$2
Current balance: $4.00
Insert more coin?(Y/N): N
-Press C to cancel- or -# to change- or -R to restart- or -Y to dispense- : Y
Add sugar? (Y/N)N
No sugar will be added to your drink.
Thank you.
Please take your "capuccino".
The remaining change in the machine is $2.50.
+++++++++++++++++++++++++++++
2023-02-13 21:39:30.457970
Product sold: capuccino
Sal